### Install (if needed) and Import Libraries 

In [1]:
import tweepy
import requests as re
import json
import sys
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import operator
#!{sys.executable} -m pip install tweepy

### Conection to Google Cloud Database (to export news dataframes)


In [2]:
driver = 'mysql+pymysql'
ip = ''
username = ''
password = ''
db = ''

cs  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(cs)

### Authentication for twitter API

In [3]:
consumer_token = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

### Getting tweets from diferent sources (timelines)

In [4]:
# twitter users used
list_pages = ['LaVanguardia', 'elperiodico', 'diariARA', 'elmundoes', 'el_pais', 'elnacionalcat']

In [5]:
# loop in order to get tweets from different user's timelines

for page in list_pages:
    tweets = api.user_timeline(screen_name=page)
    
    print(page, ' start')
    # generate series with the content of the tweets
    date = pd.Series([tweet._json['created_at'] for tweet in tweets])
    text = pd.Series([tweet._json['text'] for tweet in tweets])
    retweets = pd.Series([tweet._json['retweet_count'] for tweet in tweets])
    likes = pd.Series([tweet._json['favorite_count'] for tweet in tweets])
    media = pd.Series([page for tweet in tweets])
    
    list_urls = []
    for tweet in tweets:
        elem = ''
        if tweet._json['entities']['urls'] != []:
            elem = tweet._json['entities']['urls'][0]['expanded_url']
            if tweet._json['entities']['urls'][0]['expanded_url'].find('https://') > -1:
                elem = elem.split('https://')[1]
            if tweet._json['entities']['urls'][0]['expanded_url'].find('?') > -1:
                elem = elem.split('?')[0]
        list_urls.append(elem) 
            
    url = pd.Series(list_urls)


    # create one dataframe with all the tweets from each source
    data = pd.DataFrame({'text': text, 'url': url, 'date': date, 'retweets': retweets, 'likes': likes, 'media': media})
    
    print(page, ' ends')
    
    # export dataframe to database with all the tweets
    data.to_sql(con=engine, name='tweets', if_exists='append')

LaVanguardia  start
LaVanguardia  ends
elperiodico  start
elperiodico  ends
diariARA  start
diariARA  ends
elmundoes  start
elmundoes  ends
el_pais  start
el_pais  ends
elnacionalcat  start
elnacionalcat  ends


### Update Dicttionary with words

In [6]:
Ssql = '''SELECT 
    MAX(tweets.`index`) AS `index`,
    MAX(`text`) AS `text`,
    MAX(url) AS url,
    MAX(`date`) AS `date`,
    MAX(retweets) AS retweets,
    MAX(likes) AS likes,
    MAX(tweets.media) AS media
FROM
    tweets
GROUP BY tweets.`text`'''

auxdf = pd.read_sql(Ssql,engine)

#### +Function that counts how many likes and interactions get EACH WORD (number of likes+retweets of the tweets where the word appears)

In [7]:
def export_words_values(field, likes, retweets):
    for word in field.split(' '):
        clean_word = word.strip('¿,.":;/?!\n ')
        if 'http' not in word:
            if clean_word in words.keys():
                words[clean_word][0] += likes+retweets
                words[clean_word][1] += 1
            else:
                words[clean_word] = [likes+retweets, 1]

#### +Initialize an empty dictionary and goes through the dataframe exporting words & interactions

In [8]:
words = {}

In [9]:
a = auxdf[['text','likes', 'retweets']].apply(lambda x: export_words_values(x['text'], x['likes'], x['retweets']), axis=1)

### Create a sorted dataframe with all the words in tweets and the amount of interactions

In [10]:
words = sorted(words.items(), key=operator.itemgetter(1), reverse=True)

In [11]:
table_words = pd.DataFrame({'word': [x[0] for x in words], 'interactions': [x[1][0] for x in words], 'appearances': [x[1][1] for x in words]}) 
table_words.head(10)

,word,interactions,appearances
0,de,23604,1226
1,la,13123,685
2,en,8849,461
3,a,7477,479
4,el,6648,455
5,y,6503,231
6,que,6384,424
7,con,5950,120
8,un,5138,228
9,del,3804,268


### Export the table to database (replace if exist)

In [12]:
table_words.to_sql(con=engine, name='words', if_exists='replace')